# 6. 작사가 AI

## 6.1 데이터를 불러오고 확인해본다

In [1]:
# 먼저 프로젝트를 진행하기 전에 아래와 같은 작업이 필요하다
# $ wget https://aiffelstaticprd.blob.core.windows.net/media/documents/song_lyrics.zip
# $ unzip song_lyrics.zip -d ~/aiffel/lyricist/data/lyrics

# 데이터를 읽어온다
import glob
import os
import re
import tensorflow as tf
import numpy as np

txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 모두 읽어서 raw_corpus 에 담는다
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ['', '\t\t\t“There must be some way out of here,” said the joker to the thief', '“There’s too much confusion, I can’t get no relief']


In [2]:
# 문장이 어떤 것이 있는지 확인해보자
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장은 건너뛴다.
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뛴다.

    if idx > 10: break   # 문장 10개를 확인해보자
        
    print(sentence)

			“There must be some way out of here,” said the joker to the thief
“There’s too much confusion, I can’t get no relief
Businessmen, they drink my wine, plowmen dig my earth
None of them along the line know what any of it is worth”
“No reason to get excited,” the thief, he kindly spoke
“There are many here among us who feel that life is but a joke
But you and I, we’ve been through that, and this is not our fate
So let us not talk falsely now, the hour is getting late”


## 6.2 데이터를 전처리한다

In [3]:
# 토큰화하기 전에 정규표현식을 이용해서 문장을 필터링 해주는 함수를 만든다
def preprocess_sentence(sentence):
    sentence = sentence.lower().strip()       # 소문자로 바꾸고 양쪽 공백을 삭제
  
    # 아래 3단계를 거쳐 sentence는 스페이스 1개를 delimeter로 하는 소문자 단어 시퀀스로 바뀐다
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)        # 패턴의 특수문자를 만나면 특수문자 양쪽에 공백을 추가
    sentence = re.sub(r'[" "]+', " ", sentence)                  # 공백 패턴을 만나면 스페이스 1개로 치환
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)  # a-zA-Z?.!,¿ 패턴을 제외한 모든 문자(공백문자까지도)를 스페이스 1개로 치환

    sentence = sentence.strip()

    sentence = '<start> ' + sentence + ' <end>'      # 문장 앞뒤로 <start>와 <end>를 단어처럼 붙여 준다
    
    return sentence

print(preprocess_sentence("This @_is ;;;sample        sentence."))   # 문장이 어떻게 필터링 됐는지 확인해보자

<start> this is sample sentence . <end>


In [4]:
# 정제 데이터를 완성한다
corpus = []

for sentence in raw_corpus:
    if len(sentence) == 0: continue
    if len(sentence.split()) >=12 : continue
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장은 건너뛴다
        
    corpus.append(preprocess_sentence(sentence))
        
corpus[:10]

['<start> there s too much confusion , i can t get no relief <end>',
 '<start> businessmen , they drink my wine , plowmen dig my earth <end>',
 '<start> no reason to get excited , the thief , he kindly spoke <end>',
 '<start> all along the watchtower , princes kept the view <end>',
 '<start> while all the women came and went , barefoot servants , too <end>',
 '<start> outside in the distance a wildcat did growl <end>',
 '<start> two riders were approaching , the wind began to howl <end>',
 '<start> once upon a time you dressed so fine <end>',
 '<start> you threw the bums a dime in your prime , didn t you ? <end>',
 '<start> people d call , say , beware doll , you re bound to fall <end>']

In [5]:
# 정제된 데이터를 토큰화하고, 단어 사전을 만들고, 데이터를 숫자로 변환한다.
def tokenize(corpus):
    # 텐서플로우에서 제공하는 Tokenizer 패키지를 생성
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12500,  # 전체 단어의 개수 
        filters=' ',    # 별도로 전처리 로직을 추가할 수 있지만, 이번에는 사용하지 않겠습니다.
        oov_token="<unk>"  # out-of-vocabulary, 사전에 없었던 단어는 어떤 토큰으로 대체할지
    )
    tokenizer.fit_on_texts(corpus)   # 우리가 구축한 corpus로부터 Tokenizer가 사전을 자동구축하게 된다

    # 이후 tokenizer를 활용하여 모델에 입력할 데이터셋을 구축한다
    tensor = tokenizer.texts_to_sequences(corpus)   # tokenizer는 구축한 사전으로부터 corpus를 해석해 Tensor로 변환한다.

    # 입력 데이터의 시퀀스 길이를 일정하게 맞추기 위한 padding  메소드를 제공한다
    # maxlen의 디폴트값은 None이다. 이 경우 corpus의 가장 긴 문장을 기준으로 시퀀스 길이가 맞춰진다.
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  

    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   62   16 ...    0    0    0]
 [   2    1    4 ...    0    0    0]
 [   2   37  595 ...    0    0    0]
 ...
 [   2    9  156 ...    0    0    0]
 [   2    8    9 ...    0    0    0]
 [   2    6 3227 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7f79bb71b3d0>


In [6]:
# 생성된 텐서 데이터를 3번째 행부터 10번째 열까지 출력해 보자
print(tensor[:3, :10])

[[   2   62   16  101  186 2633    4    5   33   15]
 [   2    1    4   43  488   13  986    4    1 1037]
 [   2   37  595   10   45 2491    4    6 2237    4]]


In [7]:
# 텐서 데이터는 모두 정수로 이우러져 있는데, 이것은 tokenizer에 구축된 단어 사전의 인덱스다. 단어 사전이 어떻게 구축되었는지 확인을 한번 해보자
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 15: break

1 : <unk>
2 : <start>
3 : <end>
4 : ,
5 : i
6 : the
7 : you
8 : and
9 : a
10 : to
11 : it
12 : me
13 : my
14 : in
15 : t


In [8]:
# 데이터 패딩 처리
src_input = tensor[:, :-1]  # tensor에서 마지막 토큰을 잘라내서 소스 문장을 생성한다. 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높다.
tgt_input = tensor[:, 1:]    # tensor에서 <start>를 잘라내서 타겟 문장을 생성한다.

print(src_input[0])
print(tgt_input[0])

[   2   62   16  101  186 2633    4    5   33   15   45   37 5623    3
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]
[  62   16  101  186 2633    4    5   33   15   45   37 5623    3    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0]


In [9]:
# 데이터셋 객체를 생성한다.
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 450
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1    # tokenizer가 구축한 단어사전 내 10000개와, 여기 포함되지 않은 0:<pad>를 포함하여 10001개

dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input)).shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((450, 29), (450, 29)), types: (tf.int32, tf.int32)>

In [10]:
# 데이터셋을 적절하게 나눠준다
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=20)

In [11]:
# 잘 나눠졌는지 확인해 보자
print("Source Train:", enc_train.shape)
print("Target Train:", dec_train.shape)

Source Train: (121675, 29)
Target Train: (121675, 29)


# 6.3 모델을 생성하고 평가해본다

In [12]:
# 모델 생성 -> 1개의 Embedding 레이어와 2개의 LSTM 레이어 그리고 1개의 Dense 레이어로 구성한다
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super(TextGenerator, self).__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 1024
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
# 모델의 shape을 살펴보자
for src_sample, tgt_sample in dataset.take(1): break
model(src_sample)

<tf.Tensor: shape=(450, 29, 12501), dtype=float32, numpy=
array([[[ 3.8185174e-04, -1.7984827e-05, -1.6679820e-04, ...,
         -1.3133374e-05,  1.7482019e-04, -1.0425178e-04],
        [ 4.2313605e-04, -3.5183923e-04, -2.5149572e-04, ...,
         -3.8682108e-04,  7.4056923e-05, -2.7054608e-05],
        [ 6.4476411e-04, -5.6856166e-04, -3.8029355e-04, ...,
         -7.2526408e-04,  9.5069270e-05, -1.8655547e-06],
        ...,
        [ 1.7740838e-05,  4.8608781e-04,  8.1505073e-04, ...,
          4.7086687e-03,  3.1954604e-03,  1.4046150e-04],
        [ 7.2095943e-05,  4.5076141e-04,  9.1457617e-04, ...,
          4.8398762e-03,  3.2978568e-03,  1.6746762e-04],
        [ 1.2260905e-04,  4.1703961e-04,  9.9909690e-04, ...,
          4.9491972e-03,  3.3874011e-03,  1.9553170e-04]],

       [[ 3.8185174e-04, -1.7984827e-05, -1.6679820e-04, ...,
         -1.3133374e-05,  1.7482019e-04, -1.0425178e-04],
        [ 8.8547991e-04, -2.9163528e-04, -4.4288874e-05, ...,
         -2.6534937e-04, 

In [14]:
# 만들어진 모델을 살펴본다
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3200256   
_________________________________________________________________
lstm (LSTM)                  multiple                  5246976   
_________________________________________________________________
lstm_1 (LSTM)                multiple                  8392704   
_________________________________________________________________
dense (Dense)                multiple                  12813525  
Total params: 29,653,461
Trainable params: 29,653,461
Non-trainable params: 0
_________________________________________________________________


In [15]:
# GPU가 사용가능 한 상태인지 확인해본다
tf.test.is_gpu_available()

Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


True

In [16]:
# 훈련 시작 (에포크를 10번 정도 돌리려 했으니 시간 관계상 3번으로 끝낸다)
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

# the save point
checkpoint_dir = os.getenv('HOME')+'/aiffel/lyricist/models2/'

cp_callback = tf.keras.callbacks.ModelCheckpoint(checkpoint_dir,
                                                 save_weights_only=True,
                                                 monitor='val_loss',
                                                 mode='auto',
                                                 save_best_only=True,
                                                 verbose=1)
print("✅")

model.compile(loss=loss, optimizer=optimizer, metrics=['accuracy'])
history_lycist=model.fit(dataset, epochs=10)
'''
history = model.fit(enc_train, 
          dec_train, 
          epochs=12,
          batch_size=400,
          validation_data=(enc_val, dec_val),
          verbose=1)
'''

✅
Epoch 1/10
337/337 [==============================] - 148s 438ms/step - loss: 1.8460 - accuracy: 0.7452
Epoch 2/10
337/337 [==============================] - 146s 434ms/step - loss: 1.4820 - accuracy: 0.7689
Epoch 3/10
337/337 [==============================] - 147s 436ms/step - loss: 1.4038 - accuracy: 0.7744
Epoch 4/10
337/337 [==============================] - 147s 437ms/step - loss: 1.3451 - accuracy: 0.7782
Epoch 5/10
337/337 [==============================] - 147s 437ms/step - loss: 1.2956 - accuracy: 0.7815
Epoch 6/10
337/337 [==============================] - 147s 437ms/step - loss: 1.2499 - accuracy: 0.7848
Epoch 7/10
337/337 [==============================] - 147s 435ms/step - loss: 1.2082 - accuracy: 0.7883
Epoch 8/10
337/337 [==============================] - 147s 435ms/step - loss: 1.1695 - accuracy: 0.7917
Epoch 9/10
337/337 [==============================] - 147s 435ms/step - loss: 1.1335 - accuracy: 0.7953
Epoch 10/10
337/337 [==============================] - 146s 43

'\nhistory = model.fit(enc_train, \n          dec_train, \n          epochs=12,\n          batch_size=400,\n          validation_data=(enc_val, dec_val),\n          verbose=1)\n'

## 6.4 모델 평가

In [23]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 일단 텐서로 변환한다.
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 텍스트를 실제로 생성할때는 루프를 돌면서 단어 하나씩 생성해야 됨
    while True:
        predict = model(test_tensor)  # 입력받은 문장의 텐서를 입력
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1]   # 모델이 예측한 마지막 단어가 새롭게 생성한 단어가 됨

        # 모델이 새롭게 예측한 단어를 입력 문장의 뒤에 붙여 줌
        test_tensor = tf.concat([test_tensor, 
                                                                 tf.expand_dims(predict_word, axis=0)], axis=-1)

        # 우리 모델이 <end>를 예측했거나, max_len에 도달하지 않았다면  while 루프를 또 돌면서 다음 단어를 예측해야 됨
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # 생성된 tensor 안에 있는 word index를 tokenizer.index_word 사전을 통해 실제 단어로 하나씩 변환
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated   # 최종적으로 모델이 생성한 자연어 문장

In [25]:
generate_text(model, tokenizer, init_sentence="<start> I love", max_len=20)

'<start> i love you , i m so bad <end> '

## 회고
한주가 정말로 빨리간다... 이번 프로젝트에서는 화요일에 진행한 프로젝트와 같은 카테고리에 있는 자연어처리를 진행했는데, 그때는 음성을, 지금은 글을 어떻게 딥러닝모델에 담고 학습을 시키는지를 대략적으로 알게 되었다. 화요일 노드와 달리 이번에는 개념이 어렵지 않았고 복습도 아주 많은 시간을 들이지는 않을 거 같다. 내가 미완성된 문장을 주면 모델이 완성된 문장을 다시 리턴해주는 것과 모델을 다시 만들고 다시 학습을 시키면 리턴된 문장이 매번 달라지는 것이 신기했다. 프로젝트를 진행하면서 딱히 어려운 점은 없었으나, 모델이 무거워서 학습시간이 오래걸렸다. 점심을 먹고 다시 모델을 돌리니 예상시간이 엄청나게 길게 잡혀서 학습을 중단하고 tf.test.is_gpu_available() 명령어로 gpu가 사용가능한 상태인지 확인했더니 flase가 떠서 노트북을 껐다가 켰다. 그랬더니 True가 뜨고 모델이 어느정도 이해가능한?? 수준의 학습속도로 돌아오더라...역시 에러를 해결하는 확실한 방법은 재부팅이 아닐까... 여튼, 루브릭에서 요구하는 모든 평가문항들을 잘 수행했으니 이번에도 3별을 받지 않을까. 아 그리고 노드에서 "10 Epoch 안에 val_loss 값을 2.2 수준으로 줄일 수 있는 모델을 설계하세요!"라고 되어 있는데 프로젝트에서 어딘가 꼬이지 않는 이상은 1 Epoch에 2.2 이하로 내려가는 것을 볼 수 있다.